In [1]:
from stanza.server import CoreNLPClient
import pickle
import pandas as pd
import spacy
from tqdm import tqdm
import requests
import numpy as np

In [2]:
# set sent from file
nlp = spacy.load('en_core_web_lg')
var = pickle.load(open('./cleaned_data_split/a0.sav', 'rb'))
doc=nlp(var[0])
sent=str(list(doc.sents)[0])
sent = sent.rstrip()

In [2]:
# set sent from customizion
sent = 'charts of patients were reviewed'

In [3]:
result=[]
url = 'http://localhost:8080/minie/query'
r = requests.post(url, data=sent)
for facts in r.json()['facts']:
    result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})

In [4]:
result2=[]
with CoreNLPClient(be_quite=False, annotators=["openie"], memory='16G', endpoint='http://localhost:1234', threads=12) as client:
    ann = client.annotate(sent,output_format='json')
    for sentence in ann['sentences']:
        for triple in sentence['openie']:
            if triple:
                result2.append({'subject':triple['subject'],'relation': triple["relation"],'object':triple['object']})

2021-10-08 00:50:21 INFO: Writing properties to tmp file: corenlp_server-0948acaa9c41491b.props
2021-10-08 00:50:21 INFO: Starting server with command: java -Xmx16G -cp C:\Users\pross\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 1234 -timeout 60000 -threads 12 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-0948acaa9c41491b.props -annotators openie -preload -outputFormat serialized


In [5]:
sent

'charts of patients were reviewed'

In [6]:
np.array(result) #minie

array([{'subject': 'charts of patients', 'relation': 'were reviewed', 'object': ''}],
      dtype=object)

In [7]:
np.array(result2) #openie

array([{'subject': 'charts', 'relation': 'were', 'object': 'reviewed'}],
      dtype=object)